In [1]:
import os
%pwd

'd:\\Kidney-Disease-classification-cnn\\research'

In [2]:
os.chdir("../")

In [3]:

%pwd

'd:\\Kidney-Disease-classification-cnn'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
import gdown
from src.cnn_classifier import logger
from src.cnn_classifier.utils.common import get_size

In [8]:
class  DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
         
    def download_file(self):
      
      try:
       dataset_url=self.config.source_URL
       zip_download_dir=self.config.local_data_file
       os.makedirs("artifacts/data_ingestion",exist_ok=True)
       logger.info(f"Downloading file from :{dataset_url} into file :{zip_download_dir}")

       file_id = dataset_url.split('/')[-2]
       prefix='https://drive.google.com/uc?/export=download&id='
       gdown.download(prefix+file_id, str(zip_download_dir))

       logger.info(f"Downloaded data from: {dataset_url} into file :{zip_download_dir} of size :{get_size(zip_download_dir)}")

      except Exception as e:
         raise e
    
    def extract_zip_file(self):
       
       unzip_path=self.config.unzip_dir
       os.makedirs(unzip_path,exist_ok=True)
       with zipfile.ZipFile(self.config.local_data_file,mode="r") as zip_ref:
           zip_ref.extractall(unzip_path)
        

In [10]:
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_integestion = DataIngestion(config=data_ingestion_config)
    data_integestion.download_file()
    data_integestion.extract_zip_file()
except Exception as e:
    raise e 

[2025-10-31 05:28:00,354 - cnn_classifier - INFO - yaml file:config\config.yaml loaded successfully]


[2025-10-31 05:28:00,379 - cnn_classifier - INFO - yaml file:params.yaml loaded successfully]
[2025-10-31 05:28:00,381 - cnn_classifier - INFO - created directory at:artifacts]
[2025-10-31 05:28:00,383 - cnn_classifier - INFO - created directory at:artifacts/data_ingestion]
[2025-10-31 05:28:00,385 - cnn_classifier - INFO - Downloading file from :https://drive.google.com/file/d/15AWxlgsbETlz277Hqc_9bt39EqIuwfNj/view?usp=sharing into file :artifacts\data_ingestion\data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=15AWxlgsbETlz277Hqc_9bt39EqIuwfNj
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=15AWxlgsbETlz277Hqc_9bt39EqIuwfNj&confirm=t&uuid=652f7f33-1bff-461a-9522-8f802d2998e2
To: d:\Kidney-Disease-classification-cnn\artifacts\data_ingestion\data.zip
100%|██████████| 940M/940M [30:25<00:00, 515kB/s]

[2025-10-31 05:58:29,399 - cnn_classifier - INFO - Downloaded data from: https://drive.google.com/file/d/15AWxlgsbETlz277Hqc_9bt39EqIuwfNj/view?usp=sharing into file :artifacts\data_ingestion\data.zip of size :~ 917945 KB]
